# Notebook for the performance test report 

This Python code is used to generate a performance report PDF from PCM/uprof monitoring data collected during readout application tests.

The key functions are:

* plot_vars_comparison(): Plots performance metrics from PCM/uprof data for multiple tests into comparison plots. It generates a plot for each socket.

* create_report_performance(): Creates the full PDF report. It:

    * Processes the raw PCM/uprof data if needed
    * Generates the comparison plots by calling plot_vars_comparison()
    * Adds intro text, table of tests, and the plots to the PDF
    * Prints CPU core pinning info for each test
    * It takes input data from a specified folder, processes it, generates plots in an output folder, and builds the PDF report with custom text, table, and plots.

How to run it:
To extract the data from a given dashboard in grafana: 
* extract_data_and_stats_from_panel(grafana_url, dashboard_uid, delta_time, host, input_dir, output_csv_file)
* 'grafana_url' is:
    * 'http://np04-srv-009.cern.ch:3000'  (legacy)
    * 'http://np04-srv-017.cern.ch:31023' (new) 
* 'dashboard_uid' is the unic dashboard identifiyer, you can find this information on the link of the dashboard. The dashboard_uid code is in the web link after/d/.../ 
    * for intel-r-performance-counter-monitor-intel-r-pcm dashboard dashboard_uid = '91zWmJEVk' 
* delta_time is [start, end] given in the format '%Y-%m-%d %H:%M:%S'
* host is the name of the server in study for example: "np02-srv-003"     
* output_csv_file (for performance tests): [version]-[server_app_tested]-[numa node]-[data format]-[tests_name]
    * example of name: v4_1_1-np02srv003-0-eth-stream_scaling

To create the the performance report: 
* create_report_performance(input_dir, output_dir, daqconfs_cpupins_folder_parent_dir, process_pcm_files=False, process_uprof_files=False, print_info=True, streams='8, 16, 24, 32, 40, and 48', pdf_name='performance_report', repin_threads_file=None, elog_message=['TBA'])
    * 
    * 

Helper functions all bdefined in basic_functions.py:

* make_name_list(): Generates lists of file names in the input folder
* break_file_name(): Parses info from a file name
* add_new_time_format(): Adds a timestamp column to PCM/uprof data
* uprof_pcm_formatter(): Converts uprof data to PCM-like format
* json_info(): Prints CPU pinning info for a test

So in summary, it automates generating a performance report from raw monitoring data, including custom intro text, test info table, comparison plots, and configuration details.

### Import the modules needed, defining paths and funtions

In [1]:
from basic_functions import *

print('Cheking list of packages need it')
for package_i in list_py_package:
    debug_missing_module(module_name=package_i)

pcm_columns_list_0 = ['C0 Core C-state residency', 'Socket0 Memory Bandwidth', 'Socket0 Instructions Per Cycle', 
                      'Socket0 L2 Cache Misses', 'Socket0 L2 Cache Hits', 
                      'Socket0 L3 Cache Misses', 'Socket0 L3 Cache Hits']

pcm_columns_list_1 = ['C0 Core C-state residency', 'Socket1 Memory Bandwidth', 'Socket1 Instructions Per Cycle', 
                      'Socket1 L2 Cache Misses', 'Socket1 L2 Cache Hits', 
                      'Socket1 L3 Cache Misses', 'Socket1 L3 Cache Hits']

uprof_columns_list_0 = [' Utilization (%) Socket0', 'Total Mem Bw (GB/s) Socket0', 'IPC (Sys + User) Socket0', 
                        'L2 Miss (pti) Socket0', 'L2 Access (pti) Socket0', 
                        'L3 Miss Socket0', 'L3 Miss % Socket0']

uprof_columns_list_1 = ['Utilization (%) Socket1', 'Total Mem Bw (GB/s) Socket1', 'IPC (Sys + User) Socket1', 
                        'L2 Miss (pti) Socket1', 'L2 Access (pti) Socket1', 
                        'L3 Miss Socket1', 'L3 Miss % Socket1']

label_names = ['CPU Utilization (%)', 'Memory Bandwidth (GB/sec)', 'Instructions Per Cycle', 
               'L2 Cache Misses (Million)', 'L2 Cache [Misses/Accesses] (%)', 
               'L3 Cache Misses (Million)', 'L3 Cache [Misses/Accesses] (%)']

label_columns = ['Socket0', 'Socket1']    

def plot_vars_comparison(input_dir, output_dir, pdf_name):
    # Function to 
    X_plot, Y_plot_0, Y_plot_1, label_plot_0, label_plot_1 = [], [], [], [], []
    
    pcm_file, uprof_file, core_utilization_file, reformated_uprof_file, reformated_core_utilization_file, all_file, all_plots_file = make_name_list(input_dir)
    
    for i, file_i in enumerate(all_plots_file):    
        info = break_file_name(file_i)
        data_frame = pd.read_csv('{}/{}.csv'.format(input_dir, file_i))
        X_plot.append(data_frame['NewTime'].values.tolist())
                
        Y_tmp_0, Y_tmp_1, label_tmp_0, label_tmp_1 = [], [], [], []
        
        if info[0]=='grafana':
            for k, (columns_pcm_0, columns_pcm_1) in enumerate(zip(pcm_columns_list_0, pcm_columns_list_1)):
                Y_0, label_0 = get_column_val(data_frame, [columns_pcm_0], [label_columns[0]], file_i)  
                Y_1, label_1 = get_column_val(data_frame, [columns_pcm_1], [label_columns[1]], file_i)  
                Y_tmp_0.append(Y_0)
                label_tmp_0.append(label_0)
                Y_tmp_1.append(Y_1)
                label_tmp_1.append(label_1)
        else:
            for k, (columns_uprof_0, columns_uprof_1) in enumerate(zip(uprof_columns_list_0, uprof_columns_list_1)):
                Y_0, label_0 = get_column_val(data_frame, [columns_uprof_0], [label_columns[0]], file_i)
                Y_1, label_1 = get_column_val(data_frame, [columns_uprof_1], [label_columns[1]], file_i)
                Y_tmp_0.append(Y_0)
                label_tmp_0.append(label_0)
                Y_tmp_1.append(Y_1)
                label_tmp_1.append(label_1)
    
        Y_plot_0.append(Y_tmp_0)
        label_plot_0.append(label_tmp_0)
        Y_plot_1.append(Y_tmp_1)
        label_plot_1.append(label_tmp_1)
    
    # Here we make the plot:
    matplotlib.rcParams['font.family'] = 'DejaVu Serif'
    fig, axs = plt.subplots(3, 1, figsize=(18, 12))
    plt.style.use('default')
    axs = axs.flatten()
    #axs[3].axis('off')
    
    for i in range(len(Y_plot_0)):  #number of files or tests
        for j in range(len(Y_plot_0[i])):  #number of metrix
            if j < 3:
                label0_ij0 = re.sub('_', ' ', label_plot_0[i][j][0])
                axs[j].plot(X_plot[i], Y_plot_0[i][j][0], color=color_list[i], label=label0_ij0, linestyle=linestyle_list[0])
                axs[j].set_ylabel('{}'.format(label_names[j]))
                axs[j].set_xlabel('Time (min)')
                axs[j].grid(which='major', color='gray', linestyle='dashed')
                axs[j].legend(loc='upper left')
            else:
                pass
                
    plt.tight_layout()
    plt.savefig('{}/{}_results_{}_{}_socket0.png'.format(output_dir, pdf_name, info[1], info[4]))
    plt.close() 
    
    fig, axs = plt.subplots(2, 2, figsize=(18, 8))
    plt.style.use('default')
    axs = axs.flatten()   
    
    for i in range(len(Y_plot_0)):  
        for j in range(len(Y_plot_0[i])):
            if j < 3:
                pass
            else:
                label0_ij0 = re.sub('_', ' ', label_plot_0[i][j][0])
                axs[j-3].plot(X_plot[i], Y_plot_0[i][j][0], color=color_list[i], label=label0_ij0, linestyle=linestyle_list[0])
                axs[j-3].set_ylabel('{}'.format(label_names[j]))
                axs[j-3].set_xlabel('Time (min)')
                axs[j-3].grid(which='major', color='gray', linestyle='dashed')
                axs[j-3].legend(loc='upper left')
                
    plt.tight_layout()
    plt.savefig('{}/{}_results_cache_{}_{}_socket0.png'.format(output_dir, pdf_name, info[1], info[4]))
    plt.close() 
    
    fig, axs = plt.subplots(3, 1, figsize=(18, 12))
    plt.style.use('default')
    axs = axs.flatten()
    
    for i in range(len(Y_plot_1)):  
        for j in range(len(Y_plot_1[i])):
            if j < 3:
                label1_ij0 = re.sub('_', ' ', label_plot_1[i][j][0])
                axs[j].plot(X_plot[i], Y_plot_1[i][j][0], color=color_list[i], label=label1_ij0, linestyle=linestyle_list[0])
                axs[j].set_ylabel('{}'.format(label_names[j]))
                axs[j].set_xlabel('Time (min)')
                axs[j].grid(which='major', color='gray', linestyle='dashed')
                axs[j].legend(loc='upper left')
            else:
                pass
    
    plt.tight_layout()
    plt.savefig('{}/{}_results_{}_{}_socket1.png'.format(output_dir, pdf_name, info[1], info[4]))
    plt.close() 
    
    fig, axs = plt.subplots(2, 2, figsize=(18, 8))
    plt.style.use('default')
    axs = axs.flatten()
    
    for i in range(len(Y_plot_1)):  
        for j in range(len(Y_plot_1[i])):
            if j < 3:
                pass
            else:
                label1_ij0 = re.sub('_', ' ', label_plot_1[i][j][0])
                axs[j-3].plot(X_plot[i], Y_plot_1[i][j][0], color=color_list[i], label=label1_ij0, linestyle=linestyle_list[0])
                axs[j-3].set_ylabel('{}'.format(label_names[j]))
                axs[j-3].set_xlabel('Time (min)')
                axs[j-3].grid(which='major', color='gray', linestyle='dashed')
                axs[j-3].legend(loc='upper left')
    
    plt.tight_layout()
    plt.savefig('{}/{}_results_cache_{}_{}_socket1.png'.format(output_dir, pdf_name, info[1], info[4]))
    plt.close() 

def create_report_performance(input_dir, output_dir, daqconfs_cpupins_folder_parent_dir, process_pcm_files=False, process_uprof_files=False, print_info=True, streams='8, 16, 24, 32, 40, and 48', pdf_name='performance_report', repin_threads_file=None, elog_message=['TBA']):    
    # Function to 
    directory([input_dir, output_dir])
    pcm_file, uprof_file, core_utilization_file, reformated_uprof_file, reformated_core_utilization_file, all_file, all_plots_file = make_name_list(input_dir)
    
    # Open pdf file
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font('Times', 'B', 16)
    pdf.cell(40,10,'Performance Report')
    pdf.ln(10)
    
    # Processing the data first
    if process_pcm_files:
        for i, file_pcm_i in enumerate(pcm_file):
            add_new_time_format(input_dir, file_pcm_i)

    if process_uprof_files:
        for i, file_uprof_i in enumerate(uprof_file):
            uprof_pcm_formatter(input_dir, file_uprof_i)
            add_new_time_format(input_dir, 'reformatter_{}'.format(file_uprof_i))
    
    cpupins_utilazation_reformatter(input_dir)
    for i, file_core_i in enumerate(core_utilization_file):
         add_new_time_format_utilization(input_dir, 'reformatter_{}'.format(file_core_i))
        
    if process_pcm_files or process_uprof_files:
        print('Finish the processing of the data.')
    
    info_pcm_basic = break_file_name(all_file[0])
    
    # creating report
    pdf.set_font('Times', '', 10)
    pdf.write(5, 'The tests were run using the dunedaq version fddaq-{} and for the WIB{} data format for {} streams. The Figures 1 and 2 show the results of the tests ran (Table1) using the different metrics. \n'.format(info_pcm_basic[1], info_pcm_basic[4], streams))
    pdf.write(5, '    * L2-hits is the fraction of requests that make it to L2 at all. Similar for L3. \n')
    pdf.write(5, '    * L2-misses is the fraction of requests that make it to L2 at all and then miss in L2. Similar for L3. \n')
    pdf.ln(10)
    
    #-------------------------------------------TABLE-----------------------------------------------
    # Data to tabular
    rows_data = []
    headers = ['Test', 'Readout app SRV', 'OS', 'NODE', 'Elog message']
    rows_data.append(headers)
    
    line_height = pdf.font_size * 2
    col_width = [pdf.epw/3.8, pdf.epw/6.8, pdf.epw/12, pdf.epw/12, pdf.epw/2.8]  
    lh_list = [] #list with proper line_height for each row
    
    for i, file_i in enumerate(all_file):
        info = break_file_name(file_i)
        test_info = re.sub('_', ' ', info[5])
        #line = [info[5], info[2], check_OS(info[2]), info[3], elog_message[i]]
        line = [test_info, info[2], check_OS(info[2]), info[3], elog_message[i]]
        rows_data.append(line)
    
    # Determine line heights based on the number of words in each cell
    for row in rows_data:
        max_lines = 1  # Initialize with a minimum of 1 line
        for datum in row:
            lines_needed = len(str(datum).split('\n'))  # Count the number of lines
            max_lines = max(max_lines, lines_needed)
 
        lh_list.append(line_height * max_lines)
        
    # Add table rows with word wrapping and dynamic line heights
    for j, row in enumerate(rows_data):
        line_height_table = lh_list[j] 
        for k, datum in enumerate(row):
            pdf.multi_cell(col_width[k], line_height_table, datum, border=1, align='L', new_x=XPos.RIGHT, new_y=YPos.TOP, max_line_height=pdf.font_size)
            
        pdf.ln(line_height_table)
        
    pdf.write(5, 'Table 1. Summary of the tests ran. \n')    
    pdf.ln(10)
    
    #--------------------------------------------FIGURES------------------------------------------------
    plot_vars_comparison(input_dir, output_dir, pdf_name)
    
    if info[3] == '0' or info[3] == '2':
        pdf.image('{}/{}_results_{}_{}_socket0.png'.format(output_dir, pdf_name, info_pcm_basic[1], info_pcm_basic[4]), w=180)
        pdf.write(5, 'Figure 1. Socket0 results of the tests ran using the metrics CPU Utilization (%), Memory Bandwidth (GB/sec), Instructions Per Cycle.')
        pdf.ln(10)
        pdf.image('{}/{}_results_cache_{}_{}_socket0.png'.format(output_dir, pdf_name, info_pcm_basic[1], info_pcm_basic[4]), w=180)
        pdf.write(5, 'Figure 2. Socket0 results of the tests ran using the metrics L2 Cache Misses (Million), L2 Cache [Misses/Hits] (%), L3 Cache Misses (Million), and L3 Cache [Misses/Hits] (%).')
        pdf.ln(10)
        
    if info[3] == '1' or info[3] == '2':
        pdf.image('{}/{}_results_{}_{}_socket1.png'.format(output_dir, pdf_name, info_pcm_basic[1], info_pcm_basic[4]), w=180)
        pdf.write(5, 'Figure 1. Socket1 results of the tests ran using the metrics CPU Utilization (%), Memory Bandwidth (GB/sec), Instructions Per Cycle.')
        pdf.ln(10)
        pdf.image('{}/{}_results_cache_{}_{}_socket1.png'.format(output_dir, pdf_name, info_pcm_basic[1], info_pcm_basic[4]), w=180)
        pdf.write(5, 'Figure 2. Socket1 results of the tests ran using the metrics L2 Cache Misses (Million), L2 Cache [Misses/Hits] (%), L3 Cache Misses (Million), and L3 Cache [Misses/Hits] (%).')
        pdf.ln(10)
    
    #----------------------------------------CONFIGURATIONS---------------------------------------------
    pcm_file, uprof_file, core_utilization_file, reformated_uprof_file, reformated_core_utilization_file, all_file, all_plots_file = make_name_list(input_dir)
    
    if print_info:
        pdf.write(5, 'Configurations: \n', 'B')
        for i, (file_i, file_ii) in enumerate(zip(all_file, reformated_core_utilization_file)):
            info = break_file_name(file_i)
            file_daqconf_i='daqconf-{}-{}-{}-{}'.format(info[4], info[5], info[2], info[3])
            var_i='ru{}{}{}'.format(info[2], info[4], '0')
            json_info(file_daqconf=file_daqconf_i, file_core=file_ii, input_directory=daqconfs_cpupins_folder_parent_dir, input_dir=input_dir, var=var_i, pdf=pdf, if_pdf=print_info, repin_threads_file=repin_threads_file)
            pdf.cell(0, 10, 'Table {}. CPU core pins information for the "{}" test.'.format(i+2, info[5]))
            pdf.ln(10)
            
    pdf.ln(20)
    pdf.set_font('Times', '', 10)
    pdf.write(5, 'The End, made on {}'.format(current_time()))
    pdf.output('{}/{}_report.pdf'.format(output_dir, pdf_name))
    
print('Ready to run and process')

Cheking list of packages need it
Ready to run and process


### Proccesing data from Grafana
Note: change the paths to fit yours

In [ ]:
# without raw recording

grafana_url = 'http://np04-srv-009.cern.ch:3000'
dashboard_uid = ['91zWmJEVk']
host_used = 'np02-srv-003'  
delta_time = [['2023-11-28 12:59:32', '2023-11-28 14:11:36'],
              ['2023-11-28 14:18:46', '2023-11-28 15:31:18']]
output_csv_file = ['v4_2_1-np02srv003-0-eth-stream_scaling', 
                   'v4_2_1-np02srv003-0-eth-stream_scaling_swtpg']
results_path = '/eos/home-d/dvargas/SWAN_projects/performance_results/without_recording'

for delta_time_list, output_csv_file_list in zip(delta_time, output_csv_file):
    extract_data_and_stats_from_panel(grafana_url, dashboard_uid, delta_time=delta_time_list, host=host_used, 
                                      input_dir=results_path, output_csv_file=output_csv_file_list)

print('done :-)')

In [ ]:
# with raw recording

grafana_url = 'http://np04-srv-009.cern.ch:3000'
dashboard_uid = ['91zWmJEVk']
host_used = 'np02-srv-003'  
delta_time = [['2023-11-28 15:38:44', '2023-11-28 16:50:50'],
              ['2023-11-29 09:59:53', '2023-11-29 11:12:27']]
output_csv_file = ['v4_2_0-np02srv003-0-eth-stream_scaling_recording', 
                   'v4_2_0-np02srv003-0-eth-stream_scaling_recording_swtpg']
results_path = '/eos/home-d/dvargas/SWAN_projects/performance_results/with_recording'

for delta_time_list, output_csv_file_list in zip(delta_time, output_csv_file):
    extract_data_and_stats_from_panel(grafana_url, dashboard_uid, delta_time=delta_time_list, host=host_used, 
                                      input_dir=results_path, output_csv_file=output_csv_file_list)

print('done :-)')

In [ ]:
# just 48 stereams fro tpg tests

grafana_url = 'http://np04-srv-009.cern.ch:3000'
dashboard_uid = ['91zWmJEVk']
host_used = 'np02-srv-003'  
tpg_delta_time = [['2023-11-23 18:41:24', '2023-11-23 18:56:32'],
                  ['2023-11-23 18:58:48', '2023-11-23 19:13:56'],
                  ['2023-11-24 13:15:46', '2023-11-24 13:30:54'],
                  ['2023-11-24 13:55:35', '2023-11-24 14:10:44'],
                  ['2023-11-24 11:55:41', '2023-11-24 12:10:50'],
                  ['2023-11-24 12:13:48', '2023-11-24 12:28:56']]
tpg_output_csv_file = ['v4_2_0-np02srv003-0-eth-48streams_swtpg', 
                       'v4_2_0-np02srv003-1-eth-48streams_swtpg', 
                       'v4_2_0-np02srv003-0-eth-48streams_recording_swtpg',
                       'v4_2_0-np02srv003-1-eth-48streams_recording_swtpg',
                       'v4_2_0-np02srv003-1-eth-48streams_recording_swtpg_multinode',
                       'v4_2_0-np02srv003-1-eth-48streams_recording_swtpg_multinode_notrigger']
tpg_results_path = '/eos/home-d/dvargas/SWAN_projects/performance_results/with_tpgs_intel'

for delta_time_list, output_csv_file_list in zip(tpg_delta_time, tpg_output_csv_file):
    extract_data_and_stats_from_panel(grafana_url, dashboard_uid, delta_time=delta_time_list, host=host_used, 
                                      input_dir=tpg_results_path, output_csv_file=output_csv_file_list)

print('done :-)')

In [ ]:
# mman spsc queue test app

grafana_url = 'http://np04-srv-009.cern.ch:3000'
dashboard_uid = ['91zWmJEVk']
host_used = 'np02-srv-003'  
delta_time = [['2024-01-25 13:57:17', '2024-01-25 14:04:27']]
output_csv_file = ['v4_2_1-np02srv003-0-eth-queuetest_Intel']
results_path = '/eos/user/m/mman/performancetest/tools/queuetest_results'

for delta_time_list, output_csv_file_list in zip(delta_time, output_csv_file):
    extract_data_and_stats_from_panel(grafana_url, dashboard_uid, delta_time=delta_time_list, host=host_used, 
                                      input_dir=results_path, output_csv_file=output_csv_file_list)

print('done :-)')


### Performance report
Note: change the paths to fit yours

In [2]:
results_path = '/eos/home-d/dvargas/SWAN_projects/performance_results/without_recording'
report_path = '/eos/home-d/dvargas/SWAN_projects/performance_results/reports'
performancetest_path = '/eos/home-d/dvargas/SWAN_projects/performancetest'

create_report_performance(input_dir=results_path, output_dir=report_path, 
                          daqconfs_cpupins_folder_parent_dir=performancetest_path, 
                          process_pcm_files=True, process_uprof_files=True, 
                          print_info=True, streams='8, 16, 24, 32, 40, and 48', 
                          pdf_name='performancetest_without_recording', 
                          elog_message=['0.1 Hz', '0.1 Hz second'])

print('THE END')

New CSV file saved as: reformatter_core_utilization-v4_2_1-np02srv003-0-eth-stream_scaling.csv 
New CSV file saved as: reformatter_core_utilization-v4_2_1-np02srv003-0-eth-stream_scaling_swtpgs.csv 
Finish the processing of the data.
THE END


In [ ]:
## mman readout queue perftest
results_path = '/eos/user/m/mman/performancetest/tools/queuetest_results'
report_path = '/eos/user/m/mman/performancetest/tools/performance_reports'
performancetest_path = '/eos/user/m/mman/performancetest'

create_report_performance(input_dir=results_path, output_dir=report_path, 
                          daqconfs_cpupins_folder_parent_dir=performancetest_path, 
                          process_pcm_files=True, process_uprof_files=True, 
                          print_info=True, streams='4', 
                          pdf_name='queuetest_AMD_debug',
                          elog_message=['Unable to push within timeout period (timeout period was 0 milliseconds)', 
                                        'Unable to push within timeout period (timeout period was 0 milliseconds)'])

print('THE END')

In [ ]:
results_path = '/eos/home-d/dvargas/SWAN_projects/performance_results/with_recording'
report_path = '/eos/home-d/dvargas/SWAN_projects/performance_results/reports'
performancetest_path = '/eos/home-d/dvargas/SWAN_projects/performancetest'

create_report_performance(input_dir=results_path, output_dir=report_path, 
                          daqconfs_cpupins_folder_parent_dir=performancetest_path, 
                          process_pcm_files=True, process_uprof_files=True, 
                          print_info=True, streams='8, 16, 24, 32, 40, and 48', 
                          pdf_name='performancetest_with_recording_report',
                          elog_message=['0.1 Hz', '0.1 Hz second'])

print('THE END')

In [ ]:
#Performance report for 48 streams only, to study the TPG rate problems in the intel

tpg_results_path = '/eos/home-d/dvargas/SWAN_projects/performance_results/with_tpgs_intel'
report_path = '/eos/home-d/dvargas/SWAN_projects/performance_results/reports'
performancetest_path = '/eos/home-d/dvargas/SWAN_projects/performancetest'

create_report_performance(input_dir=tpg_results_path, output_dir=report_path, 
                          daqconfs_cpupins_folder_parent_dir=performancetest_path, 
                          process_pcm_files=True, process_uprof_files=False, 
                          print_info=True, streams='48', pdf_name='performancetest_tpg_intel_report',
                          elog_message=['0.1 Hz', '0.1 Hz second'])

print('THE END')

In [ ]:
#Performance report for 48 streams only, to study the TPG rate problems in the amd

tpg_results_path = '/eos/home-d/dvargas/SWAN_projects/performance_results/with_tpgs_amd'
report_path = '/eos/home-d/dvargas/SWAN_projects/performance_results/reports'
performancetest_path = '/eos/home-d/dvargas/SWAN_projects/performancetest'

create_report_performance(input_dir=tpg_results_path, output_dir=report_path, 
                          daqconfs_cpupins_folder_parent_dir=performancetest_path, 
                          process_pcm_files=False, process_uprof_files=True, 
                          print_info=True, streams='48', pdf_name='performancetest_tpg_amd_report',
                          elog_message=['0.1 Hz', '0.1 Hz second'])

print('THE END')